In [79]:
import pandas as pd
import numpy as np
import math
from math import sin, cos, sqrt, atan2
from matplotlib import pyplot as plt
import numpy as np
from statsmodels.formula.api import ols
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.feature_selection import SelectKBest, f_regression,mutual_info_regression
from sklearn.feature_selection import RFECV
import warnings
import seaborn as sns
import pickle
from sklearn.preprocessing import StandardScaler
plt.style.use('seaborn')

# warnings.filterwarnings("ignore")

pd.set_option('display.max_columns', 100)

In [80]:
df=pd.read_csv('survey.csv')

In [81]:
df.columns=[x.lower() for x in df.columns]

In [82]:
gender_comp= pd.DataFrame(df.gender)

In [83]:
gender_comp

,gender
0,Female
1,M
2,Male
3,Male
4,Male
...,...
1254,male
1255,Male
1256,male
1257,f


In [84]:
conditions=[
    (df.gender.str.lower()=='male')|(df.gender.str.lower()=='m'),
    (df.gender.str.lower()=='female')|(df.gender.str.lower()=="f")|(df.gender.str.lower()=="woman"),
    (df.gender.str.lower()=='mail')|(df.gender.str.lower()=='msle')|(df.gender.str.lower()=='maile')|(df.gender.str.lower()=='mal')
    
]
choises=[
    'Male',
    'Female',
    "Male",
    
]
df['gender']=pd.DataFrame(np.select(conditions,choises,default="Gender Variant"))

In [85]:
gender_comp2=pd.DataFrame(df.gender)

In [86]:
gender=pd.concat([gender_comp,gender_comp2],axis=1)

In [87]:
gender

,gender,gender
0,Female,Female
1,M,Male
2,Male,Male
3,Male,Male
4,Male,Male
...,...,...
1254,male,Male
1255,Male,Male
1256,male,Male
1257,f,Female


In [88]:
conditions=[
    df.no_employees=="1-5",            
    df.no_employees=="6-25",         
    df.no_employees=="26-100", 
    df.no_employees=="100-500",       
    df.no_employees=="500-1000" ,           
    df.no_employees=="More than 1000",
]
choises=[
    5,
    25,
    100,
    500,
    1000,
    10000,
    
]
df['no_employees']=np.select(conditions,choises,default=0)

In [89]:
def to_num(column):
    """
    Takes in a column name as string value. Converts all string yes, no, maybe and dont know as int values.
        to_num('is_employed')
    """
    conditions=[
        (df[column].str.lower()=='yes'),            
        (df[column].str.lower()=='no'),
        (df[column].str.lower()=='maybe')|(df[column].str.lower()=='don\'t know')|(df[column].str.lower()=='not sure'),
    ]
    choises=[
        1,
        0,
        2,
    ]
    df[column]=np.select(conditions,choises,default=None)

In [90]:
cols=["self_employed","family_history","treatment","remote_work","tech_company","benefits","care_options","wellness_program","seek_help","anonymity","phys_health_consequence","mental_health_interview","phys_health_interview","mental_vs_physical","obs_consequence"]
for col in cols:
    to_num(str(col))
    

In [91]:
conditions=[
    (df["mental_health_consequence"].str.lower()=='yes'),            
    (df["mental_health_consequence"].str.lower()=='no'),
    (df["mental_health_consequence"].str.lower()=='maybe')|(df["mental_health_consequence"].str.lower()=='don\'t know')|(df["mental_health_consequence"].str.lower()=='not sure'),
]
choises=[
    1,
    0,
    1,

]
df["mental_health_consequence"]=np.select(conditions,choises,default=0)

In [92]:
df.mental_health_consequence.value_counts()

1    769
0    490
Name: mental_health_consequence, dtype: int64

In [93]:
df.leave.value_counts()

Don't know            563
Somewhat easy         266
Very easy             206
Somewhat difficult    126
Very difficult         98
Name: leave, dtype: int64

In [94]:
"  \"  "

'  "  '

In [95]:
print("  \"  ")

  "  


In [96]:
df

,timestamp,age,gender,country,state,self_employed,family_history,treatment,work_interfere,no_employees,remote_work,tech_company,benefits,care_options,wellness_program,seek_help,anonymity,leave,mental_health_consequence,phys_health_consequence,coworkers,supervisor,mental_health_interview,phys_health_interview,mental_vs_physical,obs_consequence,comments
0,2014-08-27 11:29:31,37,Female,United States,IL,None,0,1,Often,25,0,1,1,2,0,1,1,Somewhat easy,0,0,Some of them,Yes,0,2,1,0,NaN
1,2014-08-27 11:29:37,44,Male,United States,IN,None,0,0,Rarely,10000,0,0,2,0,2,2,2,Don't know,1,0,No,No,0,0,2,0,NaN
2,2014-08-27 11:29:44,32,Male,Canada,NaN,None,0,0,Rarely,25,0,1,0,0,0,0,2,Somewhat difficult,0,0,Yes,Yes,1,1,0,0,NaN
3,2014-08-27 11:29:46,31,Male,United Kingdom,NaN,None,1,1,Often,100,0,1,0,1,0,0,0,Somewhat difficult,1,1,Some of them,No,2,2,0,1,NaN
4,2014-08-27 11:30:22,31,Male,United States,TX,None,0,0,Never,500,1,1,1,0,2,2,2,Don't know,0,0,Some of them,Yes,1,1,2,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1254,2015-09-12 11:17:21,26,Male,United Kingdom,NaN,0,0,1,NaN,100,0,1,0,0,0,0,2,Somewhat easy,0,0,Some of them,Some of them,0,0,2,0,NaN
1255,2015-09-26 01:07:35,32,Male,United States,IL,0,1,1,Often,100,1,1,1,1,0,0,1,Somewhat difficult,0,0,Some of them,Yes,0,0,1,0,NaN
1256,2015-11-07 12:36:58,34,Male,United States,CA,0,1,1,Sometimes,10000,0,1,1,1,0,0,2,Somewhat difficult,1,1,No,No,0,0,0,0,NaN
1257,2015-11-30 21:25:06,46,Female,United States,NC,0,0,0,NaN,500,1,1,0,1,0,0,2,Don't know,1,0,No,No,0,0,0,0,NaN


In [97]:
for col in cols:
    print(df[col].value_counts())

0    1095
1     146
Name: self_employed, dtype: int64
0    767
1    492
Name: family_history, dtype: int64
1    637
0    622
Name: treatment, dtype: int64
0    883
1    376
Name: remote_work, dtype: int64
1    1031
0     228
Name: tech_company, dtype: int64
1    477
2    408
0    374
Name: benefits, dtype: int64
0    501
1    444
2    314
Name: care_options, dtype: int64
0    842
1    229
2    188
Name: wellness_program, dtype: int64
0    646
2    363
1    250
Name: seek_help, dtype: int64
2    819
1    375
0     65
Name: anonymity, dtype: int64
0    925
2    273
1     61
Name: phys_health_consequence, dtype: int64
0    1008
2     207
1      44
Name: mental_health_interview, dtype: int64
2    557
0    500
1    202
Name: phys_health_interview, dtype: int64
2    576
1    343
0    340
Name: mental_vs_physical, dtype: int64
0    1075
1     184
Name: obs_consequence, dtype: int64


In [99]:
df[df.work_interfere.isna()].groupby('gender').count()

,timestamp,age,country,state,self_employed,family_history,treatment,work_interfere,no_employees,remote_work,tech_company,benefits,care_options,wellness_program,seek_help,anonymity,leave,mental_health_consequence,phys_health_consequence,coworkers,supervisor,mental_health_interview,phys_health_interview,mental_vs_physical,obs_consequence,comments
gender,,,,,,,,,,,,,,,,,,,,,,,,,,
Female,35,35,35,26,35,35,35,0,35,35,35,35,35,35,35,35,35,35,35,35,35,35,35,35,35,5
Gender Variant,5,5,5,3,5,5,5,0,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,1
Male,224,224,224,113,224,224,224,0,224,224,224,224,224,224,224,224,224,224,224,224,224,224,224,224,224,15


In [100]:
df.work_interfere.value_counts()

Sometimes    465
Never        213
Rarely       173
Often        144
Name: work_interfere, dtype: int64

In [101]:
df.work_interfere=df.work_interfere.fillna("Missing")

In [102]:
w_interfere=df[['work_interfere','gender','age']].groupby(['gender','work_interfere']).count()

In [103]:
w_interfere['prop']=round((w_interfere/w_interfere.groupby(level=0).sum())*100,2)

In [104]:
w_interfere.columns=['count','prop']

In [110]:
w_interfere.to_csv('WI_gender.csv')

In [112]:
w_interfere.drop('count',axis=1,inplace=True)